In [1]:
import pandas as pd
import requests as req
import plotly.express as px
from datetime import datetime
import dash


**Visualisation of top 100 starred repositories on github.com**

In [2]:
name_list = []
stars_list = []
url_list = []
repos_list = []
forkcount_list = []
overall_count = 0
repo_creation_date = []

index = 1

In [60]:
while overall_count < 100:
    repos = req.get('https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page='+str(index)+ '&per_page=100&rel="next"').json()
    for repo in repos['items']:
        name_list.append(repo['name'])
        stars_list.append(repo['stargazers_count'])
        url_list.append(repo['html_url'])
        repos_list.append(repo['full_name'])
        forkcount_list.append(repo['forks_count'])
        #append the date and times to the list
        repo_creation_date.append(datetime.strptime(repo['created_at'], '%Y-%m-%dT%H:%M:%SZ'))
        overall_count += 1
    index += 1


In [26]:
#make a scatter plot with lists: name_list, stars_list, forkcount_list
fig = px.scatter(x=stars_list, y=forkcount_list, labels={'x':'Stars', 'y':'Forks'}, title='Stars vs Forks', hover_name=name_list)
fig.show()

In [27]:
#bar graph with lists: name_list, stars_list
fig = px.bar(x=name_list, y=stars_list, labels={'x':'Repository', 'y':'Stars'}, title='Stars per Repository')
fig.show()


In [29]:
#bar graph with lists: name_list, forkcount_list
fig = px.bar(x=name_list, y=forkcount_list, labels={'x':'Repository', 'y':'Forks'}, title='Forks per Repository')
fig.show()


**Getting pull request data of octocat/hello-world repository**

In [2]:
repo_name = 'tensorflow/tensorflow'
repos = req.get('https://api.github.com/repos/'+repo_name+'/pulls?q=per_page=100').json()
df_pulls = pd.DataFrame(repos)

In [3]:
#make a loop to get all the pull requests till page 3 from the api
for i in range(2,10):
    repos = req.get('https://api.github.com/repos/'+repo_name+'/pulls?q=per_page=100&page='+str(i)).json()
    df_pulls = df_pulls.append(pd.DataFrame(repos))

In [4]:
date_time = df_pulls['created_at']
id = df_pulls['id']
title = df_pulls['title']
#make a list of the dates from date_time
date = []
for i in date_time:
    date.append(datetime.strptime(i, '%Y-%m-%dT%H:%M:%SZ').date())

times = []
for i in date_time:
    times.append(datetime.strptime(i, '%Y-%m-%dT%H:%M:%SZ').time())




In [5]:
#make a scatterplot of the date versus time with a slider for date and time
fig = px.scatter(x=date, y=times, labels={'x':'Date', 'y':'Time'}, title='Date and Time of Pull Requests')
fig.update_layout(xaxis=dict(
    rangeslider_visible=True,
    rangeselector=dict(
        x=0.1,
        y=1.1,
        bgcolor='rgba(150, 200, 250, 0.4)',
        buttons=list([
            dict(count=1,
                 label="1m",
                 step="month",
                 stepmode="backward"),
            dict(count=6,
                 label="6m",
                 step="month",
                 stepmode="backward"),
            dict(count=1,
                 label="YTD",
                 step="year",
                 stepmode="todate"),
            dict(count=1,
                 label="1y",
                 step="year",
                 stepmode="backward"),
            dict(step="all")
        ])
    )
))
fig.show()